**European and American Options: An Overview**

Options are financial derivatives that offer the right, but not the obligation, to buy or sell an asset at a predetermined price, referred to as the strike price, on or before a certain date. European options can only be exercised at expiration, while American options can be exercised any time before or on the expiration date.

**Methods for Pricing European Equity and Currency Options:**

1. **Black-Scholes-Merton Model**: 
    - This is the most well-known model for pricing European options.
    - The model was initially developed for equity options but has since been adapted for currency options.
    - The model considers several factors, including the current stock or currency price, option strike price, time until expiration, implied volatility, risk-free interest rate, and, for equity options, expected dividends.
    - While it's very influential and widely used, its main limitation is that it assumes constant volatility and interest rates, which isn't always realistic in real-world markets.

2. **Monte Carlo Simulation**:
    - Uses random number generation to simulate various paths of stock or currency prices until the option's expiration.
    - Though computationally intensive, it is very flexible and can be used for complex derivatives and path-dependent options.

3. **Analytical Approximations**:
    - Employed for options that do not have closed-form solutions like the Black-Scholes.
    - These methods provide an estimate based on mathematical approximations.

4. **Jump Diffusion Models**:
    - Incorporate sudden "jumps" in prices, capturing events such as market crashes or major news impacts.
  
5. **Stochastic Volatility Models**:
    - Unlike the constant volatility assumption in Black-Scholes, these models treat volatility as a variable.
    - They capture the observed volatility smile or skew in market prices.

6. **Local Volatility Models (Dupire Model)**:
    - Derive a volatility surface from the observed market option prices, adjusting dynamically based on stock price and time.

7. **Bachelier Model**:
    - Especially relevant for currency options, this model assumes that the underlying follows a normal process, as opposed to the log-normal assumption of Black-Scholes.

**Methods for Pricing American Equity and Currency Options:**

1. **Binomial Tree Model**:
    - This method involves breaking down the option's life into a series of discrete intervals or steps.
    - In each step, the price of the underlying asset is modeled to either move up or down.
    - The option value is then determined at each node, starting from the end and moving backward to the present.
    - As the number of steps increases, this model approaches the Black-Scholes price for European options.

2. **Trinomial Tree Model**:
    - An extension of the binomial model. Here, the underlying asset price can move up, down, or remain the same in each interval, leading to a more accurate pricing mechanism.

3. **Finite Difference Methods**:
    - These methods solve the option pricing partial differential equation directly.
    - They are used for American options because of the early exercise feature which makes analytical solutions difficult.

4. **Least Squares Monte Carlo**:
    - An option pricing technique that handles the American option's early exercise feature by simulating multiple asset paths and then determining the optimal exercise strategy.

**Comparison:**

- European options, due to their restriction on exercise timing, are typically easier and less computationally intensive to price than American options.
  
- American options are often valued higher than their European counterparts (assuming all other factors equal) because they provide more flexibility to the holder.

- Currency options are affected by factors such as interest rate differentials between the two currencies involved and require modifications in standard equity option models to account for these.

In [87]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.linalg import solve

In [88]:
#Europe

def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")
        
    return option_price

In [89]:
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.20

call_price = black_scholes(S, K, T, r, sigma, option_type='call')
put_price = black_scholes(S, K, T, r, sigma, option_type='put')

print(f"Call option price: {call_price:.2f}")
print(f"Put option price: {put_price:.2f}")

Call option price: 10.45
Put option price: 5.57


In [90]:
#USA

def binomial_american_option(S, K, T, r, sigma, N, option_type='call'):
    dt = T/N
    discount = np.exp(-r * dt)
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    q = (np.exp(r*dt) - d) / (u - d)
    binomial_tree = np.zeros([N+1, N+1])
    
    for j in range(N+1):
        if option_type == 'call':
            binomial_tree[j, N] = max(0, S*(u**j)*(d**(N-j)) - K)
        elif option_type == 'put':
            binomial_tree[j, N] = max(0, K - S*(u**j)*(d**(N-j)))
    
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            if option_type == 'call':
                binomial_tree[j, i] = max(0, S*(u**j)*(d**(i-j)) - K, discount * (q*binomial_tree[j+1, i+1] + (1-q)*binomial_tree[j, i+1]))
            elif option_type == 'put':
                binomial_tree[j, i] = max(0, K - S*(u**j)*(d**(i-j)), discount * (q*binomial_tree[j+1, i+1] + (1-q)*binomial_tree[j, i+1]))
                
    return binomial_tree[0, 0]

In [91]:
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.20
N = 100  # Steps for tree-based methods; adjust for accuracy vs. performance
M = 1000  # Paths for Monte Carlo; adjust for accuracy vs. performance

for option_type in ['call', 'put']:
    binomial_price = binomial_american_option(S, K, T, r, sigma, N, option_type)
    print(f"Binomial American Option {option_type} Price: {binomial_price:.2f}")

Binomial American Option call Price: 10.43
Binomial American Option put Price: 6.08


In [92]:
def implied_volatility(price, S, K, T, r, option_type='call'):
    def loss_function(sigma):
        return black_scholes(S, K, T, r, sigma, option_type) - price
    try:
        return brentq(loss_function, 1e-5, 5)
    except ValueError:
        return np.nan

In [93]:
S = 100
K = 100
T = 1
r = 0.05
sigma_original = 0.2
option_type = 'call'

# Рассчитываем цену опциона с известной волатильностью
option_price = black_scholes(S, K, T, r, sigma_original, option_type)

# Теперь пытаемся восстановить волатильность
implied_sigma = implied_volatility(option_price, S, K, T, r, option_type)

print(f"Original sigma: {sigma_original}")
print(f"Implied sigma: {implied_sigma}")

Original sigma: 0.2
Implied sigma: 0.19999999999999998


In [94]:
def compute_d1_d2(S, K, T, r, sigma):
    """Helper function to compute d1 and d2 used in the Black-Scholes formula."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

def bs_greeks(S, K, T, r, sigma, option_type='call'):
    """Compute the Black-Scholes Greeks: Delta, Gamma, Vega, Theta, and Rho."""

    if option_type not in ['call', 'put']:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    d1, d2 = compute_d1_d2(S, K, T, r, sigma)
    
    if d1 is np.nan or d2 is np.nan:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    n_d1 = norm.pdf(d1)

    if option_type == 'call':
        delta = N_d1
        theta = (-sigma * S * n_d1) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * N_d2
        rho = K * T * np.exp(-r * T) * N_d2
    else:
        delta = N_d1 - 1
        theta = (-sigma * S * n_d1) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * (1 - N_d2)
        rho = -K * T * np.exp(-r * T) * (1 - N_d2)
    
    gamma = n_d1 / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * n_d1

    return delta, gamma, vega, theta, rho

def binomial_greeks(S, K, T, r, sigma, N, option_type='call'):
    # Изначальное значение опциона
    base_value = binomial_american_option(S, K, T, r, sigma, N, option_type)

    # Дельта
    dS = 0.01 * S
    new_value_S_up = binomial_american_option(S+dS, K, T, r, sigma, N, option_type)
    delta = (new_value_S_up - base_value) / dS

    # Гамма
    new_value_S_down = binomial_american_option(S-dS, K, T, r, sigma, N, option_type)
    gamma = (new_value_S_up - 2*base_value + new_value_S_down) / dS**2

    # Вега
    dv = 0.01
    new_value_v_up = binomial_american_option(S, K, T, r, sigma+dv, N, option_type)
    vega = (new_value_v_up - base_value) / dv

    # Тета
    dt = 0.01
    new_value_T_down = binomial_american_option(S, K, T-dt, r, sigma, N, option_type)
    theta = (new_value_T_down - base_value) / dt

    # Ро
    dr = 0.01
    new_value_r_up = binomial_american_option(S, K, T, r+dr, sigma, N, option_type)
    rho = (new_value_r_up - base_value) / dr

    return delta, gamma, vega, theta, rho

In [95]:
def test_greeks():
    S = 100
    K = 100
    T = 1
    r = 0.05
    sigma = 0.2
    N = 100

    print("Black-Scholes Greeks for Call Option:")
    delta, gamma, vega, theta, rho = bs_greeks(S, K, T, r, sigma, option_type='call')
    print(f"Delta: {delta}, Gamma: {gamma}, Vega: {vega}, Theta: {theta}, Rho: {rho}")

    print("Black-Scholes Greeks for Put Option:")
    delta, gamma, vega, theta, rho = bs_greeks(S, K, T, r, sigma, option_type='put')
    print(f"Delta: {delta}, Gamma: {gamma}, Vega: {vega}, Theta: {theta}, Rho: {rho}")

    print("Binomial Greeks for Call Option:")
    delta, gamma, vega, theta, rho = binomial_greeks(S, K, T, r, sigma, N, option_type='call')
    print(f"Delta: {delta}, Gamma: {gamma}, Vega: {vega}, Theta: {theta}, Rho: {rho}")

    print("Binomial Greeks for Put Option:")
    delta, gamma, vega, theta, rho = binomial_greeks(S, K, T, r, sigma, N, option_type='put')
    print(f"Delta: {delta}, Gamma: {gamma}, Vega: {vega}, Theta: {theta}, Rho: {rho}")

test_greeks()

Black-Scholes Greeks for Call Option:
Delta: 0.6368306511756191, Gamma: 0.018762017345846895, Vega: 37.52403469169379, Theta: -6.414027546438196, Rho: 53.232481545376345
Black-Scholes Greeks for Put Option:
Delta: -0.3631693488243809, Gamma: 0.018762017345846895, Vega: 37.52403469169379, Theta: -1.6578804239346256, Rho: -41.89046090469506
Binomial Greeks for Call Option:
Delta: 0.673941982120553, Gamma: 0.0748600395118455, Vega: 37.476277292585536, Theta: -6.414547759949585, Rho: 53.88308092625067
Binomial Greeks for Put Option:
Delta: -0.38580000414341153, Gamma: 0.054783070173799686, Vega: 37.48411606661692, Theta: -2.2441759611859347, Rho: -29.120377728491853


In [96]:
df = pd.read_excel('Options_call_put.xlsx')

# Constants
r = 0.072
T = 70 / 365.0  # Convert days to years
S_stock = 91.1
S_currency = 89.3
N = 100  # For binomial tree

# Calculation
results = []
for _, row in df.iterrows():
    strike = row['Strike']

    # European Stock Option
    european_stock_call = black_scholes(S_stock, strike, T, r, 0.20, 'call')
    european_stock_put = black_scholes(S_stock, strike, T, r, 0.20, 'put')
    
    # American Stock Option
    american_stock_call = binomial_american_option(S_stock, strike, T, r, 0.20, N, 'call')
    american_stock_put = binomial_american_option(S_stock, strike, T, r, 0.20, N, 'put')

    # European Currency Option
    european_currency_call = black_scholes(S_currency, strike, T, r, 0.20, 'call')
    european_currency_put = black_scholes(S_currency, strike, T, r, 0.20, 'put')
    
    # American Currency Option
    american_currency_call = binomial_american_option(S_currency, strike, T, r, 0.20, N, 'call')
    american_currency_put = binomial_american_option(S_currency, strike, T, r, 0.20, N, 'put')

    results.append([european_stock_call, european_stock_put, american_stock_call, american_stock_put, european_currency_call, european_currency_put, american_currency_call, american_currency_put])

results_df = pd.DataFrame(results, columns=['European Stock Call', 'European Stock Put', 'American Stock Call', 'American Stock Put', 'European Currency Call', 'European Currency Put', 'American Currency Call', 'American Currency Put'])

merged_df = pd.concat([df, results_df], axis=1)

merged_df.head()

,Call: bid,Call: offer,Strike,Put: bid,Put: offer,European Stock Call,European Stock Put,American Stock Call,American Stock Put,European Currency Call,European Currency Put,American Currency Call,American Currency Put
0,8999.0,305020.0,60.0,27,84.0,31.922800,5.102464e-07,31.922800,3.125495e-07,30.122801,0.000002,30.122801,0.000001
1,8699.0,295040.0,60.5,27,248.0,31.429657,8.424038e-07,31.429657,5.467317e-07,29.629659,0.000003,29.629658,0.000002
2,NaN,NaN,65.0,40,113.0,26.991411,4.533997e-05,26.991403,3.711702e-05,25.191487,0.000121,25.191466,0.000101
3,NaN,NaN,65.5,37,108.0,26.498290,6.690433e-05,26.498278,5.592893e-05,24.698397,0.000175,24.698377,0.000156
4,NaN,NaN,66.0,37,103.0,26.005177,9.775432e-05,26.005163,8.465747e-05,24.205330,0.000251,24.205299,0.000222


In [97]:
# Initialize lists to store the Greeks
european_stock_call_greeks = []
european_stock_put_greeks = []
american_stock_call_greeks = []
american_stock_put_greeks = []
european_currency_call_greeks = []
european_currency_put_greeks = []
american_currency_call_greeks = []
american_currency_put_greeks = []

for _, row in df.iterrows():
    strike = row['Strike']

    # Compute Greeks for each option type
    european_stock_call_greeks.append(bs_greeks(S_stock, strike, T, r, 0.20, 'call'))
    european_stock_put_greeks.append(bs_greeks(S_stock, strike, T, r, 0.20, 'put'))
    american_stock_call_greeks.append(binomial_greeks(S_stock, strike, T, r, 0.20, N, 'call'))
    american_stock_put_greeks.append(binomial_greeks(S_stock, strike, T, r, 0.20, N, 'put'))
    european_currency_call_greeks.append(bs_greeks(S_currency, strike, T, r, 0.20, 'call'))
    european_currency_put_greeks.append(bs_greeks(S_currency, strike, T, r, 0.20, 'put'))
    american_currency_call_greeks.append(binomial_greeks(S_currency, strike, T, r, 0.20, N, 'call'))
    american_currency_put_greeks.append(binomial_greeks(S_currency, strike, T, r, 0.20, N, 'put'))

# Create dataframes for each Greek set
columns_greeks = ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']
df_european_stock_call_greeks = pd.DataFrame(european_stock_call_greeks, columns=columns_greeks)
df_european_stock_put_greeks = pd.DataFrame(european_stock_put_greeks, columns=columns_greeks)
df_american_stock_call_greeks = pd.DataFrame(american_stock_call_greeks, columns=columns_greeks)
df_american_stock_put_greeks = pd.DataFrame(american_stock_put_greeks, columns=columns_greeks)
df_european_currency_call_greeks = pd.DataFrame(european_currency_call_greeks, columns=columns_greeks)
df_european_currency_put_greeks = pd.DataFrame(european_currency_put_greeks, columns=columns_greeks)
df_american_currency_call_greeks = pd.DataFrame(american_currency_call_greeks, columns=columns_greeks)
df_american_currency_put_greeks = pd.DataFrame(american_currency_put_greeks, columns=columns_greeks)

# Add Greeks to merged_df
all_greeks = [df_european_stock_call_greeks, df_european_stock_put_greeks, df_american_stock_call_greeks, df_american_stock_put_greeks, df_european_currency_call_greeks, df_european_currency_put_greeks, df_american_currency_call_greeks, df_american_currency_put_greeks]
prefixes = ['European Stock Call', 'European Stock Put', 'American Stock Call', 'American Stock Put', 'European Currency Call', 'European Currency Put', 'American Currency Call', 'American Currency Put']

for df_greek, prefix in zip(all_greeks, prefixes):
    for col in columns_greeks:
        merged_df[f'{prefix} {col}'] = df_greek[col]

merged_df.head()

,Call: bid,Call: offer,Strike,Put: bid,Put: offer,European Stock Call,European Stock Put,American Stock Call,American Stock Put,European Currency Call,...,American Currency Call Delta,American Currency Call Gamma,American Currency Call Vega,American Currency Call Theta,American Currency Call Rho,American Currency Put Delta,American Currency Put Gamma,American Currency Put Vega,American Currency Put Theta,American Currency Put Rho
0,8999.0,305020.0,60.0,27,84.0,31.922800,5.102464e-07,31.922800,3.125495e-07,30.122801,...,0.999999,3.120479e-07,0.000278,-4.262350,11.338165,-6.216993e-07,3.059238e-07,0.000280,-0.000058,-0.000012
1,8699.0,295040.0,60.5,27,248.0,31.429657,8.424038e-07,31.429657,5.467317e-07,29.629659,...,0.999999,1.074694e-06,0.000403,-4.297897,11.432642,-8.931782e-07,1.101962e-06,0.000406,-0.000086,-0.000019
2,NaN,NaN,65.0,40,113.0,26.991411,4.533997e-05,26.991403,3.711702e-05,25.191487,...,0.999959,4.613473e-05,0.013460,-4.620708,12.282133,-4.084997e-05,4.713469e-05,0.013594,-0.003250,-0.000884
3,NaN,NaN,65.5,37,108.0,26.498290,6.690433e-05,26.498278,5.592893e-05,24.698397,...,0.999931,2.774677e-05,0.017667,-4.658757,12.376164,-7.058112e-05,2.715614e-05,0.017955,-0.005863,-0.001334
4,NaN,NaN,66.0,37,103.0,26.005177,9.775432e-05,26.005163,8.465747e-05,24.205330,...,0.999906,7.014365e-05,0.025917,-4.695644,12.470106,-9.450668e-05,7.294574e-05,0.026221,-0.007162,-0.001873


In [98]:
for prefix in ['Call', 'Put']:
    merged_df[f'European Stock {prefix} Implied Volatility'] = merged_df.apply(lambda row: implied_volatility(row[f'European Stock {prefix}'], S_stock, row['Strike'], T, r, prefix.lower()), axis=1)
    merged_df[f'American Stock {prefix} Implied Volatility'] = merged_df.apply(lambda row: implied_volatility(row[f'American Stock {prefix}'], S_stock, row['Strike'], T, r, prefix.lower()), axis=1)
    merged_df[f'European Currency {prefix} Implied Volatility'] = merged_df.apply(lambda row: implied_volatility(row[f'European Currency {prefix}'], S_currency, row['Strike'], T, r, prefix.lower()), axis=1)
    merged_df[f'American Currency {prefix} Implied Volatility'] = merged_df.apply(lambda row: implied_volatility(row[f'American Currency {prefix}'], S_currency, row['Strike'], T, r, prefix.lower()), axis=1)

merged_df.head()

,Call: bid,Call: offer,Strike,Put: bid,Put: offer,European Stock Call,European Stock Put,American Stock Call,American Stock Put,European Currency Call,...,American Currency Put Theta,American Currency Put Rho,European Stock Call Implied Volatility,American Stock Call Implied Volatility,European Currency Call Implied Volatility,American Currency Call Implied Volatility,European Stock Put Implied Volatility,American Stock Put Implied Volatility,European Currency Put Implied Volatility,American Currency Put Implied Volatility
0,8999.0,305020.0,60.0,27,84.0,31.922800,5.102464e-07,31.922800,3.125495e-07,30.122801,...,-0.000058,-0.000012,0.2,0.196380,0.2,0.197049,0.2,0.196468,0.2,0.197132
1,8699.0,295040.0,60.5,27,248.0,31.429657,8.424038e-07,31.429657,5.467317e-07,29.629659,...,-0.000086,-0.000019,0.2,0.196710,0.2,0.196823,0.2,0.196766,0.2,0.196885
2,NaN,NaN,65.0,40,113.0,26.991411,4.533997e-05,26.991403,3.711702e-05,25.191487,...,-0.003250,-0.000884,0.2,0.197807,0.2,0.197775,0.2,0.197904,0.2,0.197894
3,NaN,NaN,65.5,37,108.0,26.498290,6.690433e-05,26.498278,5.592893e-05,24.698397,...,-0.005863,-0.001334,0.2,0.197894,0.2,0.198455,0.2,0.198049,0.2,0.198613
4,NaN,NaN,66.0,37,103.0,26.005177,9.775432e-05,26.005163,8.465747e-05,24.205330,...,-0.007162,-0.001873,0.2,0.198251,0.2,0.198302,0.2,0.198368,0.2,0.198435
